In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline

classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base",
                      device="cpu",
                      top_k=None)

classifier("I love this.")

Device set to use cpu


[[{'label': 'joy', 'score': 0.981096088886261},
  {'label': 'neutral', 'score': 0.006237093824893236},
  {'label': 'surprise', 'score': 0.004065955989062786},
  {'label': 'sadness', 'score': 0.0033815482165664434},
  {'label': 'anger', 'score': 0.0024693261366337538},
  {'label': 'disgust', 'score': 0.0023382133804261684},
  {'label': 'fear', 'score': 0.0004116914060432464}]]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548399329185486},
  {'label': 'neutral', 'score': 0.1698525846004486},
  {'label': 'sadness', 'score': 0.11640949547290802},
  {'label': 'surprise', 'score': 0.020700670778751373},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161491930484772},
  {'label': 'anger', 'score': 0.003935153596103191}]]

In [6]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296022176742554},
  {'label': 'neutral', 'score': 0.14038582146167755},
  {'label': 'fear', 'score': 0.06816232949495316},
  {'label': 'joy', 'score': 0.04794264957308769},
  {'label': 'anger', 'score': 0.009156368672847748},
  {'label': 'disgust', 'score': 0.002628475660458207},
  {'label': 'sadness', 'score': 0.0021221640054136515}],
 [{'label': 'neutral', 'score': 0.4493701756000519},
  {'label': 'disgust', 'score': 0.2735918462276459},
  {'label': 'joy', 'score': 0.10908295959234238},
  {'label': 'sadness', 'score': 0.09362742304801941},
  {'label': 'anger', 'score': 0.040478307753801346},
  {'label': 'surprise', 'score': 0.026970168575644493},
  {'label': 'fear', 'score': 0.006879052147269249}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.24273331463336945},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.0283005703240633},
  {'label': 'joy', 'score': 0.01421149

In [7]:
import numpy as np
emotion_labels  = ["anger", "disgust", "fear","joy", "sadness", "surprise", "neutral"]
isbn = []

emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key =lambda x : x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [9]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [10]:
emotion_scores

{'anger': [0.0641336515545845,
  0.6126195192337036,
  0.0641336515545845,
  0.3514842689037323,
  0.08141250908374786,
  0.23222456872463226,
  0.5381842255592346,
  0.0641336515545845,
  0.3006702661514282,
  0.0641336515545845],
 'disgust': [0.2735918462276459,
  0.3482843339443207,
  0.10400673747062683,
  0.15072248876094818,
  0.1844957321882248,
  0.7271750569343567,
  0.15585477650165558,
  0.10400673747062683,
  0.2794811725616455,
  0.1779264211654663],
 'fear': [0.9281684160232544,
  0.942527711391449,
  0.9723208546638489,
  0.3607058823108673,
  0.09504346549510956,
  0.051362842321395874,
  0.7474277019500732,
  0.4044981896877289,
  0.9155241250991821,
  0.051362842321395874],
 'joy': [0.932798445224762,
  0.7044222354888916,
  0.7672381401062012,
  0.2518811821937561,
  0.04056442156434059,
  0.04337584972381592,
  0.872565746307373,
  0.04056442156434059,
  0.04056442156434059,
  0.04056442156434059],
 'sadness': [0.6462159156799316,
  0.887939453125,
  0.5494765639305

In [12]:
from tqdm import tqdm

emotion_labels  = ["anger", "disgust", "fear","joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [05:23<00:00, 16.07it/s]


In [13]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [14]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612620,0.348284,0.942528,0.704422,0.887939,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081413,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759457,0.951104,0.368111,0.078765,9788185300535


In [16]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [17]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612620,0.348284,0.942528,0.704422,0.887939,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351484,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081413,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759457,0.951104,0.368111,0.078765


In [19]:
books.to_csv("books_with_emotions.csv", index=False)